In [7]:
% run items.py
import pandas as pd
import numpy as np


In [8]:
items_df = pd.DataFrame(items)
items_df

,border,color,form,letter
0,False,blue,square,E
1,True,red,circle,A
2,False,green,circle,B
3,True,red,diamond,E
4,True,red,diamond,E
5,False,blue,diamond,C
6,False,green,square,A
7,True,green,diamond,D
8,True,silver,star,C
9,False,silver,square,C


In [9]:
dat = items_df.drop('border', axis=1)
tar = items_df['border']

dat_ar = np.array(dat)
tar_ar = np.array(tar)

In [10]:
cols = list(dat.columns)

In [11]:
tree_builder(dat_ar, tar_ar, cols)

best split: form 

best split: color 

best split: letter 



IndexError: list index out of range

In [ ]:
class DecisionTree():
    
    def entropy(self, prob):
        '''
        
        '''
        # replacing - values with 1 to avoid issues with np.log2 returning - inf
        # both 0 and 1 represent pure splits so this change will not effect the result

        zero_mask = prob == 0
        prob[zero_mask] = 1

        log_2_prob = np.log2(prob)
        return -prob * log_2_prob

    def entropy_split(self, data, target, feature, debug=False):
    
#   splitting data along a feature
        unique_class, class_counts = np.unique(data[:,feature], return_counts=True)
    
        weights = class_counts/ target.shape[0]
    
        if debug: print('Weights\n{}\n'.format(weights))
    # getting the split proportions
        splits = [target[data[:,feature] == uc] for uc in unique_class]
    
        prob = np.empty((unique_class.shape[0],2))
        
        for i, split in enumerate(splits):

            prob[i][0] = split.mean()
            prob[i][1] = 1 - split.mean()

        if debug: print('Probability\n{}\n'.format(prob))
        
        entropies = self.entropy(prob)
    
        if debug: print('entropies\n{}\n'.format(entropies))
    
        # transposing entropies shape so it can be multiplied with weights
        w_ent = entropies.T * weights
            
        return w_ent.sum()
    
    def find_best_split(self, data, target, split_candidates):
        split_entropies = [self.entropy_split(data, target, split_candidate) for split_candidate in split_candidates]
    
        splt_ent = np.array(split_entropies)
        sort_ent_indx = splt_ent.argsort()
        best_ent_indx = sort_ent_indx[0]

        return split_candidates[best_ent_indx]

    def tree_builder(self, data, target, split_candidates=None):
        
        if split_candidates is None:
            split_candidates = np.array([i for i in range(data.shape[1])])
        
        total_count = target.shape[0]
        true_count = target.sum()
        
        false_count = total_count - true_count 
    
        if true_count == 0:
            return False
        elif false_count == 0:
            return True

        if split_candidates.shape[0] == 0:
            return true_count > false_count

        best_split = self.find_best_split(data, target, split_candidates)
    
        split_candidates = np.array([split_candidate for split_candidate in split_candidates if split_candidate != best_split])
 
        nodes = np.unique(data[:, best_split])
        subtree_dict = dict()
    
        for node in nodes:
            sub_mask = data[:, best_split] == node
            data_sub = data[sub_mask]
            tar_sub = target[sub_mask]
        
            subtree_dict[node] = self.tree_builder(data_sub, tar_sub, split_candidates)
    
        subtree_dict[None] = true_count > false_count
    
        return(self.features[best_split], subtree_dict)

In [4]:
def entropy(prob):
    # replacing 0 values with 1 to avoid issues with np.log2 returning - inf
    # 1 and 0 result in the same entropy value
    zero_mask = prob == 0 
    prob[zero_mask] = 1
    
    log_2_prob = np.log2(prob)
    return -prob * log_2_prob  

In [3]:
def entropy_split(data, target, feature, debug=False):
    
#   splitting data along a feature
    unique_class, class_counts = np.unique(data[:,feature], return_counts=True)
    
    weights = class_counts/ target.shape[0]
    
    if debug: print('Weights\n{}\n'.format(weights))
    # getting the split proportions
    splits = [target[data[:,feature] == uc] for uc in unique_class]
    
    prob = np.empty((unique_class.shape[0],2))
        
    for i, split in enumerate(splits):

        prob[i][0] = split.mean()
        prob[i][1] = 1 - split.mean()

    if debug: print('Probability\n{}\n'.format(prob))
        
    entropies = entropy(prob)
    
    if debug: print('entropies\n{}\n'.format(entropies))
    
    # transposing entropies shape so it can be multiplied with weights
    w_ent = entropies.T * weights
            
    return w_ent.sum()

In [2]:
def find_best_split(data, target):
    
    split_entropies = []
    
    for i in range(data.shape[1]):
        ent = entropy_split(data, target, i)
        split_entropies.append(ent)
    
    splt_ent = np.array(split_entropies)
    sort_ent_indx = splt_ent.argsort()
    best_ent_indx = sort_ent_indx[0]
    return best_ent_indx

In [14]:
def tree_builder(data, target, split_candidates):

    split_count = len(split_candidates)
    
    total_count = target.shape[0]
    true_count = target.sum()
    false_count = total_count - true_count 
    
    if not split_candidates:
        return true_count > false_count
        
    best_split_indx = find_best_split(data, target)
    best_split = split_candidates[best_split_indx]
    
    if true_count == 0:
        return False
    elif false_count == 0:
        return True
         
    new_candidates = [split_candidate for split_candidate in split_candidates if split_candidate != best_split]
    print('new_candidates: {}'.format(new_candidates))
    nodes = np.unique(data[:,best_split_indx])
    
    subtree_dict = dict()
    
    for node in nodes:
        sub_mask = data[:, best_split_indx] == node
        data_sub = data[sub_mask]
        tar_sub = target[sub_mask]
        
        subtree_dict[node] = tree_builder(data_sub, tar_sub, split_candidates=new_candidates)
    
    subtree_dict[None] = true_count > false_count
    
    print(count)
        
    return(best_split, subtree_dict)  

In [1]:
def tree_builder(data, target,split_candidates):
    
    # needs changed 
    split_count = len(split_candidates)
    
    total_count = target.shape[0]
    true_count = target.sum()
    false_count = total_count - true_count

    
    if len(split_candidates) == 0:
        return true_count > false_count
        print('done')
    
    best_split_indx = find_best_split(data, target)
    
    best_split = split_candidates[best_split_indx]
    
    print('best split: {} \n'.format(best_split))
    
    if true_count == 0: return False
    if false_count == 0: return True
    
    # removes best_split from split _candidates
    split_candidates = [split_candidate
                    for split_candidate in split_candidates
                    if split_candidate != best_split]
    
    nodes = np.unique(data[:,best_split_indx])
    
    
    subtree_dict = dict()
    
    for node in nodes:
        sub_mask = data[:,best_split_indx] == node
        data_sub = data[sub_mask]
        tar_sub = target[sub_mask]
        
        subtree_dict[node] = tree_builder(data_sub, tar_sub, split_candidates)
    
    subtree_dict[None] = true_count > false_count
        
    return(best_split, subtree_dict)